# Function / Tool Calling

In [3]:
import os
import requests
import json

In [4]:
# A function to make a call to openweathermap and get current weather

def get_weather(city_name):

    owmkey=os.environ['OWM']
    api = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={owmkey}"

    response = requests.get(api)
    return response.content

In [5]:
get_weather("mumbai")

b'{"cod":401, "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."}'